In [2]:
# coding: utf-8
from typing import Dict, List

import pandas as pd


class Datos:
    datos: int
    nominalAtributos: List[bool]

    # Constructor: procesar el fichero para asignar correctamente las variables nominalAtributos, datos y diccionarios
    def __init__(self, nombreFichero: str):
        self.datosCrudos = pd.read_csv(nombreFichero)
        self.datos = self.datosCrudos.copy()

        self.nominalAtributos = []
        self.diccionarios = {}

        for columna in self.datos.columns:
            if self._es_nominal(columna):
                self.nominalAtributos.append(True)
                self.diccionarios[columna] = self._generar_mapeo(columna)
                self.datos[columna] = self._reemplazar_valores(columna)
            elif self._es_numerico(columna):
                self.nominalAtributos.append(False)
                self.diccionarios[columna] = {}
            else:
                raise ValueError(
                    f"La columna '{columna}' contiene valores que no son nominales ni enteros/decimales."
                )

    def _es_nominal(self, columna: str) -> bool:
        # es verdadero si es la columna objetivo o si sus valores son nominales
        return columna.lower() == "class" or self.datos[columna].dtype.name == "object"

    def _es_numerico(self, columna: str) -> bool:
        # es verdadero si los valores son un n�meros enteros o reales
        return self.datos[columna].dtype.name in ["int64", "float64"]

    def _generar_mapeo(self, columna_nominal: str):
        # se extraen los valores �nicos de la columna y se sortean lexicograficamente
        valores = [str(valor) for valor in self.datos[columna_nominal].unique()]
        valores = sorted(valores)

        return {valor: indice for indice, valor in enumerate(valores)}

    def _reemplazar_valores(self, columna_nominal: str) -> pd.Series:
        mapeo = self.diccionarios[columna_nominal]
        return self.datos[columna_nominal].map(lambda valor: mapeo[str(valor)])

    # Devuelve el subconjunto de los datos cuyos �ndices se pasan como argumento
    def extraeDatos(self, idx: List[int]):
        return self.datos.iloc[idx]


In [14]:
import math
import random
import numpy as np
datos = Datos("wdbc.csv").datos


# normalizar los datos
for col in datos.columns:
    datos[col] = datos[col] / datos[col].abs().max()

#datos.head(5)

# crea umbral
umbral = [random.uniform(-0.5, 0.5) for _ in datos.columns[:-1]]


# constante de aprendizaje
k_apre = 1

# entra w.x
def sigma(w_v, x_v):
    x = np.array(x_v)[0]
    w = np.array(w_v)[0]
    #print(w, x)
    return 1 / 1 + math.exp(-w*x)


def actualiza_w(w, x):
    t = x.iloc[0,-1]
    print(t)
    x = np.array(x.iloc[:,:-1])[0]
    #print(w)
    return w - x * (sigma(w, x) - t) * k_apre

print(actualiza_w(umbral, datos.head(1)))

#print(np.array(datos.head(1))[0])


1.0
[-0.49621712 -0.44500636 -0.25030787 -0.62836922 -1.15171193 -0.45249632
 -0.81985106 -0.27593976 -1.13342254 -1.04648238 -0.81678125  0.12482717
 -0.38688682 -0.71687647 -0.49488106 -0.03645067 -0.54529522 -0.45079693
 -0.12666531 -0.35776754 -0.53435906 -0.78948426 -0.63896908 -0.69845545
 -0.92245464 -0.20145167 -0.53148158 -1.08972032 -0.43543948 -0.79623205]
